In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Qt5Agg')
%matplotlib qt
from scipy.optimize import curve_fit
import glob
import re
from tkinter import filedialog as fd
from tkinter import *
plt.rcParams['text.usetex'] = True

In [ ]:
def getFilePathFromDialog(_initialdir):
    root = Tk()
    FilePath = fd.askopenfilename(initialdir = _initialdir)
    root.destroy()
    return FilePath

def getDirectoryPathFromDialog(_initialdir):
    root = Tk()
    Directory = fd.askdirectory(initialdir = _initialdir)
    root.destroy()
    return Directory

In [ ]:
def readFile(path, skiprows):
    return np.loadtxt(path, dtype=float, skiprows=skiprows)

def readFileFromDialog(skiprows):
    return readFile(getFilePathFromDialog(dataPath),skiprows)

In [ ]:
def set_size(width, fraction=1):
    """ Set aesthetic figure dimensions to avoid scaling in latex.

    Parameters
    ----------
    width: float
            Width in pts
    fraction: float
            Fraction of the width which you wish the figure to occupy

    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    # Width of figure
    fig_width_pt = width * fraction

    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    golden_ratio = (5 ** 0.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio

    return fig_width_in, fig_height_in

In [ ]:
def readParticleConfig(path,ConfigurationNumber,Skiplines):
    TotalNumberOfParticles = int(re.search(r'(?<=N=)[\d]*',path).group())
    with open(path,'r') as F:
        Data = F.read()
    RowOffset = Skiplines+ConfigurationNumber*(TotalNumberOfParticles+2)
    print("RowOffset: ",RowOffset)
    Data = Data.strip()
    FirstRow = Data.split(sep='\n')[RowOffset]
    boxlength = float(re.search(r'(?<=BoxLength: )[\d]*.[\d]*',FirstRow).group())
    NumberOfAParticles = int(FirstRow.split()[7].strip('|'))
    NumberOfBParticles = int(FirstRow.split()[9].strip('|'))
    ListOfRows = Data.split(sep='\n')[RowOffset+1:RowOffset+1+TotalNumberOfParticles]
    if (len(ListOfRows) > 0):
        NumberOfColumns = len(ListOfRows[0].split()) - 1
    else:
        NumberOfColumns = 0
    APositions = np.empty((NumberOfAParticles, 2))
    AVelocities = np.empty((NumberOfAParticles, 2))
    BPositions = np.empty((NumberOfBParticles, 2))
    BVelocities = np.empty((NumberOfBParticles, 2))
    AIndex = 0
    BIndex = 0
    
    for i in range(len(ListOfRows)):
        if (ListOfRows[i].split()[NumberOfColumns] == 'A'):
            for j in range(2):
                APositions[AIndex,j] = ListOfRows[i].split()[j]
            for j in range(2):
                AVelocities[AIndex,j] = ListOfRows[i].split()[2+j]
            AIndex += 1
        else:
            for j in range(2):
                BPositions[BIndex,j] = ListOfRows[i].split()[j]
            for j in range(2):
                BVelocities[BIndex,j] = ListOfRows[i].split()[2+j]
            BIndex += 1
    return APositions*boxlength, AVelocities*boxlength, BPositions*boxlength, BVelocities*boxlength

def readParticleConfigFromDialog(ConfigurationNumber,Skiplines):
    return readParticleConfig(getFilePathFromDialog(dataPath),ConfigurationNumber,Skiplines)

In [ ]:
def LinearFunction(x,m,n):
    return m*x+n

In [ ]:
def extractAngles(structureFactorData):
    angleList = []
    numberOfEntriesList = []
    currentAngle = structureFactorData[0,0]
    angleList.append(currentAngle)
    numberOfEntries=0
    for angle in structureFactorData[:,0]:
        if (angle != currentAngle):
            angleList.append(angle)
            currentAngle = angle
            numberOfEntriesList.append(numberOfEntries)
            numberOfEntries=1
        else:
            numberOfEntries+=1
    numberOfEntriesList.append(numberOfEntries)
    return angleList,numberOfEntriesList

In [ ]:
def extractSeparateAngleArrays(structureFactorData):
    angleList, numberOfEntriesList = extractAngles(structureFactorData)
    angleArrayList = []
    currentIndex = 0
    for i in range(len(angleList)):
        angleArrayList.append(structureFactorData[currentIndex:currentIndex+numberOfEntriesList[i],1:])
        currentIndex+=numberOfEntriesList[i]
    return angleList,angleArrayList

In [ ]:
def plotStructureFactors(angleList,angleArrayList,angleIndex):
    fig, ax = plt.subplots(4,1,squeeze=False,figsize=(10,12), dpi = 100)
    labels = ['$S_{AA}$','$S_{BB}$','$S_{AB}$','$S_{cc}$']
    fig.suptitle('angle={}'.format(angleList[angleIndex]))
    for i in range(4):
        ax[i,0].plot(angleArrayList[angleIndex][:,0],angleArrayList[angleIndex][:,i+1])
        ax[i,0].set_ylabel(labels[i])
        ax[i,0].set_xlabel('k')
    fig.tight_layout()

In [ ]:
dataPath = getDirectoryPathFromDialog('')

## Energy series

In [ ]:
EnergySeries = readFileFromDialog(1)

In [ ]:
Fig, Ax = plt.subplots(2,2,squeeze=False,figsize=(15,9))
Ax[0,0].plot(EnergySeries[:1000,2],label='H=U+T')
Ax[0,0].plot(EnergySeries[:1000,0],label='U')
Ax[0,0].plot(EnergySeries[:1000,1],label='T')
Ax[0,0].legend()
Ax[0,1].plot(EnergySeries[:,2],label='H=U+T')
Ax[0,1].set_ylabel('H=T+U')
MeanH = np.mean(EnergySeries[:,2])
print("H=",MeanH,'+-',np.std(EnergySeries[:,2]))
print("U=",np.mean(EnergySeries[:,0]),'+-',np.std(EnergySeries[:,0]))
print("T=",np.mean(EnergySeries[:,1]),'+-',np.std(EnergySeries[:,1]))
#Ax[0,1].plot(MeanH*np.ones(len(EnergySeries[:,2])))
#Ax[1,1].vlines(MeanH,0,np.max(n),color='darkorange')
Ax[1,1].plot(EnergySeries[:,3])
Ax[1,1].set_title(r"$T_y$")
Ax[1,0].plot(EnergySeries[:,1],color='g')
Ax[1,0].set_ylabel('T')

In [ ]:
PotEnergySeriesMC = readFileFromDialog(1)

In [ ]:
Fix, Ax = plt.subplots(2,1,squeeze=False,figsize=(10,12))
Ax[0,0].plot(PotEnergySeriesMC[5000:,1])
Ax[0,0].plot(EnergySeries[5000:,1])
print("U Monte Carlo = ",np.mean(PotEnergySeriesMC[5000:,1]),"+-",np.std(PotEnergySeriesMC[5000:,1]))
print("U MD = ",np.mean(EnergySeries[5000:,1]),"+-",np.std(EnergySeries[5000:,1]))
a = Ax[1,0].hist(PotEnergySeriesMC[5000:,1],bins = 41, density = True, alpha = 0.7)
a = Ax[1,0].hist(EnergySeries[5000:,1],bins = 41, density = True,alpha =0.7)

In [ ]:
energySeries = readFileFromDialog(1)

In [ ]:
fig, ax = plt.subplots(4,1,squeeze=False,figsize=(10,12))
lowerIndex = 25000
labelList = ['U','T','H','$T_y$']
for i in range(4):
    mean = np.mean(energySeries[lowerIndex:,i])
    std = np.std(energySeries[lowerIndex:,i])
    z = ax[i,0].hist(energySeries[lowerIndex:,i],bins = 41, density = True,alpha =0.7, label='mean: {:.1f} +- {:.1f}'.format(mean,std))
    ax[i,0].legend()
    ax[i,0].set_title(labelList[i])

In [ ]:
energySeriesCorrect = readFileFromDialog(1)
energySeriesOld = readFileFromDialog(1)
energySeriesNew = readFileFromDialog(1)

In [ ]:
fig, ax = plt.subplots(4,1,squeeze=False,figsize=(10,12))
labelList = ['U','T','H','T_y']
for i in range(4):
    meanCorrect = np.mean(energySeriesCorrect[5000:,i])
    stdCorrect = np.std(energySeriesCorrect[5000:,i])
    meanNew = np.mean(energySeriesNew[5000:,i])
    stdNew = np.std(energySeriesNew[5000:,i])
    meanOld = np.mean(energySeriesOld[5000:,i])
    stdOld = np.std(energySeriesOld[5000:,i])
    z = ax[i,0].hist(energySeriesCorrect[5000:,i],bins = 41, density = True,alpha =0.7, label='correct: {:.1f} +- {:.1f}'.format(meanCorrect,stdCorrect))
    z = ax[i,0].hist(energySeriesOld[5000:,i],bins = 41, density = True,alpha =0.7, label='old: {:.1f} +- {:.1f}'.format(meanOld,stdOld))
    z = ax[i,0].hist(energySeriesNew[5000:,i],bins = 41, density = True,alpha =0.7, label='new: {:.1f} +- {:.1f}'.format(meanNew,stdNew))
    ax[i,0].legend()
    ax[i,0].set_title(labelList[i])

In [ ]:
energyDist = readFileFromDialog(1)

In [ ]:
fig, ax = plt.subplots(1,1,squeeze=False,figsize=set_size(388),dpi=300)
ax[0,0].plot(energyDist[:,0],energyDist[:,1],'.')

In [ ]:
def logOfK(x,a,b,c):
    return a*np.log(x)+b*x+c

In [ ]:
energyDistWithoutBoundaries = energyDist[8:-8,:]
popt, pcov = curve_fit(logOfK, energyDistWithoutBoundaries[:,0], np.log(energyDistWithoutBoundaries[:,1]))
print(popt)
print("T={:.5f}".format(-1.0/popt[1]))
print("N={:d}".format(round(2*(popt[0]+1))))

In [ ]:
fig, ax = plt.subplots(1,1,squeeze=False,figsize=set_size(388),dpi=300)
ax[0,0].plot(energyDistWithoutBoundaries[:,0], np.log(energyDistWithoutBoundaries[:,1]),'.')
ax[0,0].plot(energyDistWithoutBoundaries[:,0], logOfK(energyDistWithoutBoundaries[:,0],*popt),label=r'Fit to $A \log(x)+Bx+C$')
ax[0,0].set_xlabel(r'$K_y$')
ax[0,0].set_ylabel(r'$\log{P(K_y)}$')
ax[0,0].legend()
fig.tight_layout()
#fig.savefig("./P(K_y)_distribution.png",format='png',dpi=300)

## Average velocity

In [ ]:
avgVelocities = readFileFromDialog(0)

In [ ]:
Fig, Ax = plt.subplots(1,2,squeeze=False,figsize=set_size(388), dpi = 300)
Ax[0,0].plot(avgVelocities[1],avgVelocities[0],'.')
Ax[0,0].set_xlabel('$<v_x>$ [$\sigma/ t^*$]')
Ax[0,0].set_ylabel('y $[\sigma]$')
popt, pcov = curve_fit(LinearFunction, avgVelocities[1], avgVelocities[0])
print('shear rate: {:.3E}'.format(1/popt[0]))
Ax[0,0].plot(avgVelocities[1],LinearFunction(avgVelocities[1],*popt),lw=1,label='linear fit')
#Ax[0,0].set_title('$<v_x>$ vs y')
Ax[0,0].ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
Ax[0,0].legend()

Ax[0,1].plot(avgVelocities[2],avgVelocities[0],'.')
Ax[0,1].set_xlabel('$<v_y>$ [$\sigma/ t^*$]')
Ax[0,1].set_ylabel('y $[\sigma]$')
#Ax[0,1].set_title('Average $<v_y>$ vs y')
Ax[0,1].ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
Fig.tight_layout()
#Fig.savefig("./avg_velocity_profiles.png",format='png',dpi=300)

## Stress plotting

In [ ]:
shearStressSeries = readFileFromDialog(1)
print(np.mean(shearStressSeries[:]))

In [ ]:
Fig, Ax = plt.subplots(1,2, squeeze=False, figsize=(12,5))
Ax[0,0].plot(shearStressSeries[10000:],alpha=0.4)
Ax[0,0].set_xlabel('timestep')
Ax[0,0].set_ylabel('shear stress')
Ax[0,0].set_title('shear stress vs time')
#Ax[0,0].set_xscale('log')
Ax[0,0].hlines(np.mean(shearStressSeries[10000:]), 0, len(shearStressSeries[10000:]), color='orange')
#Fig.savefig("shear_stress_time_series_shear_rate=0.05.png",format='png')
n,bins,patches = Ax[0,1].hist(shearStressSeries[10000:],bins = 41, density = True)
Ax[0,1].vlines(np.mean(shearStressSeries),0,np.max(n),color='darkorange')
Ax[0,1].set_title('shear stress histogram')
Ax[0,1].set_xlabel('shear stress')

In [ ]:
NumberOfSubdivisions = 14

In [ ]:
xStressData = readFileFromDialog(2)

In [ ]:
Fig, Ax = plt.subplots(NumberOfSubdivisions,2,squeeze=False, figsize= (10,2.5*NumberOfSubdivisions),dpi =100)
xAxis = np.linspace(1.0/(2.0*NumberOfSubdivisions),1.0-1.0/(2.0*NumberOfSubdivisions),num = NumberOfSubdivisions)
for i in range(NumberOfSubdivisions):
    Ax[i,0].plot(xAxis,xStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,3],label='y={}'.format(xStressData[NumberOfSubdivisions-1-i,0]))
    Ax[i,0].plot(xAxis,np.ones(len(xAxis))*np.mean(xStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,3]))
    Ax[i,0].legend()
    Ax[i,1].plot(xAxis,xStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,4],label='y={}'.format(xStressData[NumberOfSubdivisions-1-i,0]))
    MeanShear = np.mean(xStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,4])
    Ax[i,1].plot(xAxis,np.ones(len(xAxis))*MeanShear,label='Avg = {:.4f}'.format(MeanShear))
    Ax[i,1].legend()
Fig.suptitle("Stresses of cuts in x direction (left column normal stresses, right column shear stresses)",y= 0.90)

In [ ]:
Mean = 0
for i in range(NumberOfSubdivisions-1):
    Mean += np.mean(xStressData[NumberOfSubdivisions-i-2::NumberOfSubdivisions,4])
print("Avg shear stress except top layer: {:.4f}".format(Mean/(NumberOfSubdivisions-1)))

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize= (10,6),dpi =100)
xAxis = np.linspace(1.0/(2.0*NumberOfSubdivisions),1.0-1.0/(2.0*NumberOfSubdivisions),num = NumberOfSubdivisions)
for i in range(NumberOfSubdivisions):
    line = Ax[0,0].plot(xAxis,xStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,4])
    MeanShear = np.mean(xStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,4])
    Ax[0,0].plot(xAxis,np.ones(len(xAxis))*MeanShear,color = line[0].get_color(), label='y={}, Avg = {:.4f}'.format(xStressData[NumberOfSubdivisions-1-i,0],MeanShear))
Ax[0,0].legend()
#Ax[0,0].set_ylim(-0.154,-0.146)
Ax[0,0].set_xlabel('x')
Ax[0,0].set_ylabel('shear stress')
Ax[0,0].set_title('dt=0.00001')

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize= (10,6),dpi =100)
xAxis = np.linspace(1.0/(2.0*NumberOfSubdivisions),1.0-1.0/(2.0*NumberOfSubdivisions),num = NumberOfSubdivisions)
for i in range(NumberOfSubdivisions):
    line = Ax[0,0].plot(xAxis,xStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,3])
    MeanShear = np.mean(xStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,3])
    Ax[0,0].plot(xAxis,np.ones(len(xAxis))*MeanShear,color = line[0].get_color(), label='y={}, Avg = {:.4f}'.format(xStressData[NumberOfSubdivisions-1-i,0],MeanShear))
Ax[0,0].legend()
#Ax[0,0].set_ylim(-0.154,-0.146)
Ax[0,0].set_xlabel('x')
Ax[0,0].set_ylabel('normal stress')
Ax[0,0].set_title('dt=0.0010')

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize= (10,6),dpi =100)
xAxis = np.linspace(1.0/NumberOfSubdivisions,1.0,num = NumberOfSubdivisions)
for i in range(NumberOfSubdivisions):
    line = Ax[0,0].plot(xAxis,xStressData[NumberOfSubdivisions*i:NumberOfSubdivisions*(i+1),4])
    MeanShear = np.mean(xStressData[NumberOfSubdivisions*i:NumberOfSubdivisions*(i+1),4])
    Ax[0,0].plot(xAxis,np.ones(len(xAxis))*MeanShear,color = line[0].get_color(),label='x={}'.format(xAxis[i]))
Ax[0,0].legend()
Ax[0,0].set_xlim(0,1)

In [ ]:
yStressData = readFileFromDialog(2)

In [ ]:
Fig, Ax = plt.subplots(NumberOfSubdivisions,2,squeeze=False, figsize= (10,2.5*NumberOfSubdivisions),dpi =100)
xAxis = np.linspace(1.0/(2.0*NumberOfSubdivisions),1.0-1.0/(2.0*NumberOfSubdivisions),num = NumberOfSubdivisions)
for i in range(NumberOfSubdivisions):
    Ax[i,0].plot(xAxis,yStressData[i::NumberOfSubdivisions,3],label='x={}'.format(yStressData[i,0]))
    MeanNormal = np.mean(yStressData[i::NumberOfSubdivisions,3])
    Ax[i,0].plot(xAxis,np.ones(len(xAxis))*MeanNormal)
    Ax[i,0].legend()
    Ax[i,1].plot(xAxis,yStressData[i::NumberOfSubdivisions,4],label='x={}'.format(yStressData[i,0]))
    MeanShear = np.mean(yStressData[i::NumberOfSubdivisions,4])
    Ax[i,1].plot(xAxis,np.ones(len(xAxis))*MeanShear,label='Avg={:.3f}'.format(MeanShear))
    Ax[i,1].legend()
Fig.suptitle("Stresses of cuts in y direction (left column normal stresses, right column shear stresses)",y= 0.90)

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize= (10,6),dpi =100)
xAxis = np.linspace(1.0/(2.0*NumberOfSubdivisions),1.0-1.0/(2.0*NumberOfSubdivisions),num = NumberOfSubdivisions)
for i in range(NumberOfSubdivisions):
    line = Ax[0,0].plot(xAxis,yStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,4])
    MeanShear = np.mean(yStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,4])
    Ax[0,0].plot(xAxis,np.ones(len(xAxis))*MeanShear,color = line[0].get_color(), label='x={}, Avg = {:.4f}'.format(yStressData[NumberOfSubdivisions-1-i,0],MeanShear))
Ax[0,0].legend()

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize= (10,6),dpi =100)
xAxis = np.linspace(1.0/NumberOfSubdivisions,1.0,num = NumberOfSubdivisions)
for i in range(NumberOfSubdivisions):
    line = Ax[0,0].plot(xAxis,yStressData[NumberOfSubdivisions*i:NumberOfSubdivisions*(i+1),4])
    MeanShear = np.mean(yStressData[NumberOfSubdivisions*i:NumberOfSubdivisions*(i+1),4])
    Ax[0,0].plot(xAxis,np.ones(len(xAxis))*MeanShear,color = line[0].get_color(),label='y={}'.format(xAxis[i]))
#Ax[0,0].legend()
Ax[0,0].set_xlim(0,1)

In [ ]:
print("Avg normal stress perp to x direction: {:.4f}".format(np.mean(xStressData[:,3])))
print("Avg normal stress perp to y direction: {:.4f}".format(np.mean(yStressData[:,3])))
print("Avg pressure: {:.4f}".format((np.mean(xStressData[:,3])+np.mean(yStressData[:,3]))*0.5))
print("Avg shear stress x direction: {:.4f}".format(np.mean(xStressData[:,4])))
print("Avg shear stress y direction: {:.4f}".format(np.mean(yStressData[:,4])))

In [ ]:
stressList = []
labelList = [r'$\Delta t = 0.0010$',r'$\Delta t = 0.0008$',r'$\Delta t = 0.0006$',r'$\Delta t = 0.0004$',r'$\Delta t = 0.0002$',r'$\Delta t = 0.0001$']
xData = np.zeros(NumberOfSubdivisions)

for i in range(6):
    newStress = readFileFromDialog(2)
    yData = np.zeros(NumberOfSubdivisions)
    for i in range(NumberOfSubdivisions):
        MeanShear = -1*np.mean(newStress[NumberOfSubdivisions-1-i::NumberOfSubdivisions,4])
        yData[i] = MeanShear
        xData[i] = newStress[NumberOfSubdivisions-1-i,0]
    stressList.append(yData)

In [ ]:
fig, ax = plt.subplots(1,1,squeeze=False, figsize=set_size(388), dpi=300)
for i in range(6):
    ax[0,0].plot(xData,stressList[i],'.-',label=labelList[i])
ax[0,0].set_xlabel(r'$y/L$')
ax[0,0].set_ylabel(r'average magnitude of normal stress $[\epsilon / \sigma^2]$')
ax[0,0].set_xticks(xData)
ax[0,0].legend()
fig.tight_layout()
fig.savefig("./local_shear_stress_dt_dependence.png",format='png',dpi=300)

## Shear stresses vs shear rates

In [ ]:
ShearRates = np.array([0.02,0.04,0.06,0.08,0.10])
ShearStresses = np.array([0.03509199889127006,0.07003522950514793,0.10447423035233762,0.1384373200171992,0.17166963597510385])

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(6,5), dpi = 100)
Ax[0,0].plot(ShearRates, ShearStresses,'.')
Ax[0,0].set_xlabel(r'shear rate '+r'[$1/\tau_0$]')
Ax[0,0].set_ylabel('shear stress '+r'[$\epsilon/\sigma^2$]')
popt, pcov = curve_fit(lambda x,m: LinearFunction(x,m,0.0), ShearRates, ShearStresses)
Ax[0,0].plot(ShearRates, LinearFunction(ShearRates, popt,0.0))
#Ax[0,0].legend()
print('viscosity= {:.4f}'.format(popt[0]), 'epsilon*tau_0 / (sigma^2)')
#Fig.savefig("avg_shear_stresses_vs_shear_rate_T=2.0.png",format='png')

## Number of particles

In [ ]:
NAHist0 = readFileFromDialog(1)

In [ ]:
NAHistshear = []
for i in range(5):
    newHistshear = readFileFromDialog(1)
    NAHistshear.append(newHistshear)

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False,figsize=(6,6), dpi = 100)
Ax[0,0].plot(NAHistequi[:,0],NAHistequi[:,1],label='equilibrium')
for i in range(5):
    Ax[0,0].plot(NAHistshear[i][:,0],NAHistshear[i][:,1])
Ax[0,0].legend()
#Ax[0,0].set_xlim(0.1,0.16)
#Ax[0,0].set_ylim(2.5,4.5)
Ax[0,0].set_xlabel('$x_A$')
Ax[0,0].set_ylabel('probability density')

In [ ]:
NAHist1 = readFileFromDialog(1)

In [ ]:
fig, ax = plt.subplots(1,1,squeeze=False,figsize=(6,6),dpi=100)
ax[0,0].plot(NAHist1[:,0],NAHist1[:,1])

In [ ]:
fig, ax = plt.subplots(1,1,squeeze=False,figsize=(6,6),dpi=100)
ax[0,0].plot(NAHist0[:,0],NAHist0[:,1],label='dt=0.00010')
ax[0,0].plot(NAHist1[:,0],NAHist1[:,1],label='dt=0.00001')
ax[0,0].legend()

## Finite size scaling

In [ ]:
dataN1000 = readFileFromDialog(1)
dataN2000 = readFileFromDialog(1)
dataN4000 = readFileFromDialog(1)
dataN8000 = readFileFromDialog(1)

In [ ]:
fig, ax = plt.subplots(1,2,squeeze=False,figsize=(12,6),dpi=100)
ax[0,0].plot(dataN1000[:,1],dataN1000[:,0],'.',label='N=1000')
ax[0,0].plot(dataN2000[:,1],dataN2000[:,0],'.',label='N=2000')
ax[0,0].plot(dataN4000[:,1],dataN4000[:,0],'.',label='N=4000')
ax[0,0].plot(dataN8000[:,1],dataN8000[:,0],'.',label='N=8000')
ax[0,0].set_xlabel('$x_A$')
ax[0,0].set_ylabel('T')
ax[0,0].legend()

ax[0,1].plot(dataN1000[:,0],dataN1000[:,2],'-',label='N=1000')
ax[0,1].plot(dataN2000[:,0],dataN2000[:,2],'-',label='N=2000')
ax[0,1].plot(dataN4000[:,0],dataN4000[:,2],'-',label='N=4000')
ax[0,1].plot(dataN8000[:,0],dataN8000[:,2],'-',label='N=8000')
ax[0,1].set_xlabel('$T$')
ax[0,1].set_ylabel('Binder cumulant')
ax[0,1].legend()

In [ ]:
fig, ax = plt.subplots(1,1,squeeze=False,figsize=set_size(388),dpi=300)
ax[0,0].plot(dataN1000[:,0],dataN1000[:,2],'.-',label='1000',lw=1)
ax[0,0].plot(dataN2000[:,0],dataN2000[:,2],'.-',label='2000',lw=1)
ax[0,0].plot(dataN4000[:,0],dataN4000[:,2],'.-',label='4000',lw=1)
ax[0,0].plot(dataN8000[:,0],dataN8000[:,2],'.-',label='8000',lw=1)
ax[0,0].set_xlabel(r'$T [\epsilon/k_B]$')
ax[0,0].set_ylabel('Binder cumulant')
ax[0,0].legend()
fig.tight_layout()
#fig.savefig("./finite_size_scaling_MD_hybrid.png",format='png',dpi=300)

## Binder cumulant and first moment vs shear rate

In [ ]:
cumulant_vs_shear_rate = readFileFromDialog(1)

In [ ]:
Fig, Ax = plt.subplots(1,2,squeeze=False,figsize=(12,6), dpi = 100)
Ax[0,0].plot(cumulant_vs_shear_rate[:,0],cumulant_vs_shear_rate[:,1])
Ax[0,1].plot(cumulant_vs_shear_rate[:,0],cumulant_vs_shear_rate[:,2])
Ax[0,0].set_xlabel('$\dot{\gamma}$')
Ax[0,1].set_xlabel('$\dot{\gamma}$')
Ax[0,0].set_ylabel('first moment xA')
Ax[0,1].set_ylabel('binder cumulant')

## Im(g_22)

In [ ]:
img22 = readFileFromDialog(1)

In [ ]:
Fig, Ax = plt.subplots(3,1,squeeze=False, figsize= (10, 12), dpi = 100)
labelList = ['Im($g_{22}^{AA}$)','Im($g_{22}^{BB}$)','Im($g_{22}^{AB}$)']
for i in range(3):
    Ax[i,0].plot(img22[:,0],img22[:,i+1])
    Ax[i,0].set_ylim(-0.4,0.3)
    Ax[i,0].set_xlabel('r [$\sigma$]')
    Ax[i,0].set_ylabel(labelList[i])
Fig.tight_layout()
#Fig.savefig("Img22_shear.png",format='png')

## structure factors

In [ ]:
structureFactors = readFileFromDialog(1)

In [ ]:
angleList,angleArrayList = extractSeparateAngleArrays(structureFactors)

In [ ]:
plotStructureFactors(angleList,angleArrayList,0)

In [ ]:
plotStructureFactors(angleList,angleArrayList,10)

## Avg MSD

In [ ]:
AvgMSDSeries = readFileFromDialog(0)

In [ ]:
Fig, Ax = plt.subplots(1,2,squeeze=False, figsize = (12,5), dpi = 100)
xdata = np.arange(1,len(AvgMSDSeries)+1)*stepsize
AvgMSD_Over_t_squared = AvgMSDSeries/(xdata*xdata)
Ax[0,0].plot(xdata,AvgMSD_Over_t_squared,'.')
popt, pcov = curve_fit(lambda x,n: LinearFunction(x,0.0,n), xdata[:9], AvgMSD_Over_t_squared[:9])
print(popt)
Ax[0,0].plot(xdata[:9],LinearFunction(xdata[:9],0.0,*popt))
Ax[0,0].set_xscale('log')
Ax[0,0].set_yscale('log')
Ax[0,0].set_xlabel('$t / \\tau$')
Ax[0,0].set_ylabel('$MSD/t^2$')

Ax[0,1].plot(xdata,AvgMSDSeries/xdata)
Ax[0,1].set_xscale('log')
Ax[0,1].set_yscale('log')
Ax[0,1].set_xlabel('$t / \\tau$')
Ax[0,1].set_ylabel('$MSD/t$')

## Plot state

In [ ]:
APos, AVel, BPos, BVel = readParticleConfigFromDialog(0,0)

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False,figsize=(5,5),dpi =200)
Host[0,0].plot(APos[:,0],APos[:,1],'.',color='b',ms=4)
Host[0,0].plot(BPos[:,0],BPos[:,1],'.',color='darkorange',ms=4)
Host[0,0].set_xlabel('x')
Host[0,0].set_ylabel('y')
#Host[0,0].set_xlim(0,1)
#Host[0,0].set_ylim(0,1)

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False,figsize=(5,5),dpi =200)
Host[0,0].plot(AVel[:,0],APos[:,1],'.',color='b',ms=4)
Host[0,0].plot(BVel[:,0],BPos[:,1],'.',color='darkorange',ms=4)
Host[0,0].set_ylabel('y')
Host[0,0].set_xlabel('$v_x$')
#Host[0,0].set_xlim(0,1)
#Host[0,0].set_ylim(0,1)

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False,figsize=(5,5),dpi =200)
Host[0,0].plot(AVel[:,1],APos[:,1],'.',color='b',ms=4)
Host[0,0].plot(BVel[:,1],BPos[:,1],'.',color='darkorange',ms=4)
Host[0,0].set_ylabel('y')
Host[0,0].set_xlabel('$v_y$')
#Host[0,0].set_xlim(0,1)
#Host[0,0].set_ylim(0,1)

In [ ]:
raw = """44.72728 44.87268 44.51762 44.91462 44.97332 44.86980 44.78518 44.73712 44.95986 44.68802 44.96684 45.00440 44.80572 44.46352"""
raw = raw.replace('\n',' ')
data = np.array([float(i) for i in raw.split(' ')])
std = np.std(data)
mean = np.mean(data)
print(mean,std)

In [ ]:
raw = """44.93422 45.07062 45.20840 44.91528 44.39690 44.88014 44.86854 44.75528 44.71360 44.46468 45.00148 45.17052 44.90270 45.08930
44.70452 45.23298 45.22444 45.05628 45.14416 45.00478 45.07452 45.05802 44.93718 45.08290 44.98708 44.80336 44.88786 44.79242
45.04336 45.16198 45.16678 45.33994 45.18808 45.34502 45.16040 44.91782 45.01664 44.92266 44.92864 45.06626 45.07900 45.25392
44.61332 45.06766 45.08846 44.75544 44.60614 45.10150 44.89832 45.11294 44.66008 44.70130 44.78018 44.80110 44.97484 44.84684
44.84824 45.26324 44.92670 45.43384 45.15558 44.93450 44.96298 45.11298 44.96254 44.71724 44.87698 45.20978 45.21674 44.64876
45.03862 45.30276 45.27554 45.20240 44.70988 44.51310 45.25796 44.97398 44.68210 44.88112 44.45170 44.62674 45.03096 44.72100
45.29660 44.79232 44.98498 44.65216 44.49942 44.90092 44.65282 44.63998 44.67776 44.60568 44.72934 45.02916 45.07716 44.85830
44.94562 44.83012 45.09696 44.91882 44.69836 44.75898 45.20642 45.00488 44.48826 44.85052 44.70604 44.81612 44.86726 44.81304
45.04750 44.72318 44.86192 45.16282 44.89946 44.78800 44.72904 44.64390 44.79914 45.20508 45.17442 44.94294 44.95960 45.09072
44.96570 44.90550 45.04372 45.26450 44.83182 44.95776 44.77770 45.01176 45.16226 44.93834 44.66048 45.11690 44.89894 44.98074
44.98556 44.81106 44.75342 44.85072 44.83264 45.14070 45.18968 44.87386 44.69326 44.87652 44.95970 44.91866 44.67200 44.72412
44.85264 45.11628 44.79662 44.93762 44.96542 44.80900 44.91300 45.14262 44.99132 44.70394 44.72830 44.47566 45.07100 44.83336
45.32138 44.96346 44.73636 44.69304 44.97398 45.67086 45.39566 45.23646 45.36038 44.98390 44.88620 44.97838 44.91162 45.04974"""
data = np.array([float(i) for i in raw.split()])
std = np.std(data)
mean = np.mean(data)
print(mean,std)